In [84]:
import cv2
import numpy as np

In [85]:
# Load the image
image = cv2.imread("images/coins3.jpeg")

# Resize the image to fit on the screen
scale_percent = 50  # Adjust this percentage to scale the image
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
image = cv2.resize(image, (width, height))

# Grayscale image cuz canny requires grayscale image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [86]:
# Apply GaussianBlur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

### A

In [87]:
# Edge detection using Canny
edges = cv2.Canny(blurred, 50, 150)

In [88]:
# Create a kernel for morphological operations
kernel = np.ones((3,3), np.uint8)

# Use dilation to thicken edges
dilated = cv2.dilate(edges, kernel, iterations=1)

# Use closing to close small gaps in edges
closed = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel, iterations=2)

In [89]:
# Find contours from the improved edges
contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# Draw contours around detected coins
output = image.copy()
cv2.drawContours(output, contours, -1, (0, 255, 0), 2)

# Display the results
cv2.imshow("Detected Edges", closed)
cv2.imshow("Detected Coins", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

### B

In [90]:
# Create a mask for segmentation
mask = np.zeros_like(gray)

# Extract and save each detected coin
for i, cnt in enumerate(contours):
    # Create an empty mask
    coin_mask = np.zeros_like(gray)

    # Draw the filled contour on the mask
    cv2.drawContours(coin_mask, [cnt], -1, 255, thickness=cv2.FILLED)

    # Bitwise-AND to extract only the coin
    segmented_coin = cv2.bitwise_and(image, image, mask=coin_mask)

    # Find bounding box and crop the coin
    x, y, w, h = cv2.boundingRect(cnt)
    cropped_coin = segmented_coin[y:y+h, x:x+w]

    # Save and display each segmented coin
    coin_filename = f"coin_{i+1}.png"
    cv2.imwrite(coin_filename, cropped_coin)
    cv2.imshow(f"Coin {i+1}", cropped_coin)

cv2.waitKey(0)
cv2.destroyAllWindows()

### C

In [91]:
# Count the number of detected coins
num_coins = len(contours)

# Display the count on the image
output_text = f"Total Coins: {num_coins}"
cv2.putText(output, output_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

# Show the final result
cv2.imshow("Final Detected Coins", output)

# Print the total count in the terminal
print("Total number of coins detected:", num_coins)

cv2.waitKey(0)
cv2.destroyAllWindows()


Total number of coins detected: 5
